# Benchmark Models

## Initialize

In [7]:
%load_ext autoreload
%autoreload 2

import os
from tqdm.auto import tqdm
import pathlib
import datetime
import subprocess
import numpy as np
import pandas as pd
import lifelines

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
base_path = "/home/jakobs"

project_path = f"{base_path}/data"

experiment = '230629'
experiment_path = f"{project_path}/{experiment}"
pathlib.Path(experiment_path).mkdir(parents=True, exist_ok=True)

partitions = [i for i in range(10)]

today = experiment 

In [9]:
endpoint_defs = pd.read_feather(f"{base_path}/data/endpoints_epic_md.feather").set_index("endpoint")
endpoints = endpoint_defs.index.to_list()

In [10]:
from scripts.benchmarks_iteration import load_data, calculate_iteration

In [11]:
os.environ['MKL_NUM_THREADS'] = "1"
os.environ['NUMEXPR_NUM_THREADS'] = "1"
os.environ['OMP_NUM_THREADS'] = "1"

In [13]:
rows_ray = []
iterations = [i for i in range(100)]
model = "RetinaUKB"
t_eval = 10
for iteration in tqdm(iterations[:10]):
    output_path, experiment_path, in_path, out_path, endpoints, scores, prediction_paths, eids_dict = load_data()
    for endpoint in tqdm(endpoints): 
        eids_e = eids_dict[endpoint]

        #ds = calculate_iteration.remote(in_path, prediction_paths, endpoint, scores, partition, model, t_eval, iteration, eids_e, output_path) #ray

        ds = calculate_iteration(in_path, prediction_paths, endpoint, scores, model, t_eval, iteration, eids_e, output_path) #ray
        rows_ray.append(ds)

        del eids_e

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

...

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

In [14]:
#rows = [ray.get(r) for r in tqdm(rows_ray)] # ray
rows = rows_ray # not ray
rows_finished = [item for sublist in rows for item in sublist]
benchmark_endpoints = pd.DataFrame(rows_finished)

benchmark_endpoints.to_feather(f"{experiment_path}/benchmarks_230629.feather")

In [15]:
benchmark_endpoints.groupby("score")["cindex"].mean().to_frame("cindex").sort_values("cindex")

,cindex
score,
Retina,0.569173
Age+Sex,0.592459
SCORE2,0.592964
ASCVD,0.595097
Age+Sex+Retina,0.602586
SCORE2+Retina,0.603018
ASCVD+Retina,0.604345


In [16]:
benchmark_endpoints.groupby("score")["cindex"].mean().to_frame("cindex").sort_values("cindex")

,cindex
score,
Retina,0.569173
Age+Sex,0.592459
SCORE2,0.592964
ASCVD,0.595097
Age+Sex+Retina,0.602586
SCORE2+Retina,0.603018
ASCVD+Retina,0.604345
